In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer  # Corrected import
from sklearn.feature_extraction.text import TfidfTransformer  # Corrected import
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


# Read dataset


In [8]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [9]:
fake.shape

(23481, 4)

In [10]:
true.shape

(21417, 4)

# Data cleaning and preparation


In [20]:
# add target data in the columns/
fake['target'] = 'fake'
true['target'] = 'true'

In [21]:
fake.head()

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake


In [22]:
true.head()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",true
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",true
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",true


In [23]:
# emeged the data fake and true/ concatenate the dataframes
data = pd.concat([fake,true]).reset_index(drop = True)

In [25]:
data.shape

(44898, 5)

In [28]:
# shuffle the data
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)

In [29]:
# Check the data
data.head()

,title,text,subject,date,target
0,Soda taxes spread after votes in four U.S. cities,(Reuters) - Three California cities voted for ...,politicsNews,"November 9, 2016",true
1,"Kentucky Republicans Introduce Abortion Ban, ...",Kentucky Republicans declared war on women as ...,News,"January 4, 2017",fake
2,White House weighs U.S. biofuels program chang...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,"March 6, 2017",true
3,U.S. and China align over North Korea sanction...,WASHINGTON (Reuters) - The White House said on...,politicsNews,"February 25, 2016",true
4,HILLARY’S “KHAN MAN”: Who Is KHIZR KHAN? The S...,Clock Boy Part II. When you peel back the rott...,politics,"Aug 1, 2016",fake


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   target   44898 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [33]:
# Remove the data
data.drop(['date'],axis = 1, inplace = True)

In [34]:
data.head()

,title,text,subject,target
0,Soda taxes spread after votes in four U.S. cities,(Reuters) - Three California cities voted for ...,politicsNews,true
1,"Kentucky Republicans Introduce Abortion Ban, ...",Kentucky Republicans declared war on women as ...,News,fake
2,White House weighs U.S. biofuels program chang...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,true
3,U.S. and China align over North Korea sanction...,WASHINGTON (Reuters) - The White House said on...,politicsNews,true
4,HILLARY’S “KHAN MAN”: Who Is KHIZR KHAN? The S...,Clock Boy Part II. When you peel back the rott...,politics,fake


In [35]:
# convert to lowercase
data['text'] = data['text'].apply(lambda x:x.lower())
data.head()

,title,text,subject,target
0,Soda taxes spread after votes in four U.S. cities,(reuters) - three california cities voted for ...,politicsNews,true
1,"Kentucky Republicans Introduce Abortion Ban, ...",kentucky republicans declared war on women as ...,News,fake
2,White House weighs U.S. biofuels program chang...,washington (reuters) - the trump administratio...,politicsNews,true
3,U.S. and China align over North Korea sanction...,washington (reuters) - the white house said on...,politicsNews,true
4,HILLARY’S “KHAN MAN”: Who Is KHIZR KHAN? The S...,clock boy part ii. when you peel back the rott...,politics,fake


In [36]:
import string

def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str
data['text'] = data['text'].apply(punctuation_removal)

In [38]:
# check
data.head()


,title,text,subject,target
0,Soda taxes spread after votes in four U.S. cities,reuters three california cities voted for a t...,politicsNews,true
1,"Kentucky Republicans Introduce Abortion Ban, ...",kentucky republicans declared war on women as ...,News,fake
2,White House weighs U.S. biofuels program chang...,washington reuters the trump administration i...,politicsNews,true
3,U.S. and China align over North Korea sanction...,washington reuters the white house said on th...,politicsNews,true
4,HILLARY’S “KHAN MAN”: Who Is KHIZR KHAN? The S...,clock boy part ii when you peel back the rotte...,politics,fake
